# ATDA 5340 - Final - Part 3 - Data preprocessing - Denis Shilkin

https://catalog.data.gov/dataset/aviation-data-and-documentation-from-the-ntsb-accident-database-system
<br>Public: This dataset is intended for public access and use. 
<br>License: No license information was provided. If this work was prepared by an officer or employee of the United States government as part of that person's official duties it is considered a U.S. Government Work. 

## 1. Import libraries

In [9]:
# import libraries 
import pandas as pd
import numpy as np

from pandas.plotting import scatter_matrix
from matplotlib import pyplot

# supress warnings
import warnings
warnings.filterwarnings("ignore")

print("Diagnostic: Libraries loaded, warnings set to \"Ignore\"")

Diagnostic: Libraries loaded, warnings set to "Ignore"


## 2. Load dataset and check its size

In [10]:
filename = "convertcsv.csv"

df = pd.read_csv(filename, index_col=False)

print(df.shape)

(84088, 31)


Dataset has 84088 records (rows).

### Top 5 rows

In [11]:
print(df.head(5))

         _EventId _InvestigationType _AccidentNumber  _EventDate  \
0  20191118X84400           Accident      GAA20CA081  11/17/2019   
1  20191116X25814           Accident      GAA20CA076  11/15/2019   
2  20191112X03916           Accident      GAA20CA067  11/12/2019   
3  20191112X40210           Accident      GAA20CA080  11/11/2019   
4  20191111X00550           Accident      GAA20CA070  11/11/2019   

            _Location       _Country  _Latitude  _Longitude _AirportCode  \
0  West Jefferson, OH  United States  39.935555  -83.236389         49OH   
1     San Antonio, TX  United States        NaN         NaN          SAT   
2  Pembroke Pines, FL  United States  26.001111  -80.240833          HWO   
3      Fitzgerald, GA  United States  31.683889  -83.270834          FZG   
4       Kalispell, MT  United States  41.240277 -120.526111         9CL3   

       _AirportName  ... _PurposeOfFlight _AirCarrier _TotalFatalInjuries  \
0               NaN  ...         Personal         NaN    

### Bottom 5 rows

In [12]:
print(df.tail(5))

             _EventId _InvestigationType _AccidentNumber  _EventDate  \
84083  20041105X01764           Accident      CHI79FA064  08/02/1979   
84084  20001218X45448           Accident      LAX96LA321  06/19/1977   
84085  20061025X01555           Accident      NYC07LA005  08/30/1974   
84086  20001218X45447           Accident      LAX94LA336  07/19/1962   
84087  20001218X45444           Accident      SEA87LA080  10/24/1948   

             _Location       _Country  _Latitude  _Longitude _AirportCode  \
84083       Canton, OH  United States        NaN         NaN          NaN   
84084       EUREKA, CA  United States        NaN         NaN          NaN   
84085    Saltville, VA  United States  36.922223  -81.878056          NaN   
84086   BRIDGEPORT, CA  United States        NaN         NaN          NaN   
84087  MOOSE CREEK, ID  United States        NaN         NaN          NaN   

      _AirportName  ... _PurposeOfFlight _AirCarrier _TotalFatalInjuries  \
84083          NaN  ...     

## 3. Data preprocess
<br> Check number of NaNs (Null, No Value, etc.)

In [13]:
# check number of NaNs
print(df.isnull().sum())

_EventId                     0
_InvestigationType           4
_AccidentNumber              0
_EventDate                   0
_Location                   76
_Country                   507
_Latitude                54039
_Longitude               54048
_AirportCode             36439
_AirportName             33735
_InjurySeverity              0
_AircraftDamage           2676
_AircraftCategory        56751
_RegistrationNumber       3778
_Make                       70
_Model                      99
_AmateurBuilt              592
_NumberOfEngines          4970
_EngineType               4280
_FARDescription          57056
_Schedule                72269
_PurposeOfFlight          4791
_AirCarrier              79927
_TotalFatalInjuries      27068
_TotalSeriousInjuries    29726
_TotalMinorInjuries      28523
_TotalUninjured          14507
_WeatherCondition         3060
_BroadPhaseOfFlight       6691
_ReportStatus                0
_PublicationDate         14236
dtype: int64


### Data fields description and analysis.
<br>_EventId 0
<br>_InvestigationType 4
<br>_AccidentNumber 0
<br>_EventDate                   0
<br>_Location                   76
<br>_Country                   507
<br>_Latitude                54039
<br>_Longitude               54048
<br>_AirportCode             36439
<br>_AirportName             33735
<br>_InjurySeverity              0
<br>_AircraftDamage           2676
<br>_AircraftCategory        56751
<br>_RegistrationNumber       3778
<br>_Make                       70
<br>_Model                      99
<br>_AmateurBuilt              592
<br>_NumberOfEngines          4970
<br>_EngineType               4280
<br>_FARDescription          57056
<br>_Schedule                72269
<br>_PurposeOfFlight          4791
<br>_AirCarrier              79927
<br>_TotalFatalInjuries      27068
<br>_TotalSeriousInjuries    29726
<br>_TotalMinorInjuries      28523
<br>_TotalUninjured          14507
<br>_WeatherCondition         3060
<br>_BroadPhaseOfFlight       6691
<br>_ReportStatus                0
<br>_PublicationDate         14236